In [1]:
from openpyxl import Workbook,load_workbook

In [2]:
wb = load_workbook("Proje1_VeriKümesiORJ.xlsx") #işlem yapılcak dosya
ws=wb["Sayfa1"] #calisilan worksheet

#newEx = Workbook()
#new2=newEx.active
#new2.title="main"

In [3]:
## Kolonlardaki Eksik Verilerin Tespiti
for kolon in range(1,13):
    c = 0
    for satir in range(1,156):
        if(ws.cell(satir,kolon).value == None ):
            c += 1 
    print(f"{ws.cell(1,kolon).value} alanindaki boş veri sayısı, {c}")

ID alanindaki boş veri sayısı, 0
Densite alanindaki boş veri sayısı, 0
Polarite alanindaki boş veri sayısı, 77
H20_Skoru alanindaki boş veri sayısı, 9
O2_Skoru alanindaki boş veri sayısı, 0
Azot_Skoru alanindaki boş veri sayısı, 11
H20_Oranı alanindaki boş veri sayısı, 7
O2_Oranı alanindaki boş veri sayısı, 2
Azot_Oranı alanindaki boş veri sayısı, 0
Kaynaklar alanindaki boş veri sayısı, 7
Isı_Oranı alanindaki boş veri sayısı, 2
Sınıf Bilgisi alanindaki boş veri sayısı, 3


In [4]:
##Sutunlardaki verileri kucukten buyuge siralayip all listesinin icerisine liste olarak atiyor.

all = []
for kolon in range(1,13):
    d = []
    for satir in range(2,156):
        if ws.cell(satir,kolon).value != None:
            d.append(ws.cell(satir,kolon).value) 
    d.sort()
    all.append(d)


In [5]:
### n.sutundaki veriler icin IQR hesabi
for n in range(len(all)-1):
    q1_index = len(all[n])/4
    #print(q1_index)
    if str(q1_index)[-2:] != ".0" :
        q1 = (all[n][int(q1_index-0.5)] + all[n][int(q1_index+0.5)])/2 
        q2 = (all[n][int(3*q1_index-0.5)] + all[n][int(3*q1_index+0.5)])/2 
    else:
        q1 = all[n][int(q1_index)]
        q2 = all[n][3*int(q1_index)] 


    IQR  = 1.5*(q2-q1)
    lower_limit =  q1 - IQR
    upper_limit =  q2 + IQR
    #print("Limitler: ",lower_limit,"  " ,upper_limit)
    
    print(f"{ws.cell(1,(n+1)).value} icin limit degerler: {lower_limit,upper_limit}")
    c = 1 
    #for i in all[n]:
        #if i < lower_limit:
            #print(i,"AAA")
        #elif i> upper_limit:
            #print(i,"AAA")

ID icin limit degerler: (-76.0, 232.0)
Densite icin limit degerler: (-5.7250000000000005, 11.175)
Polarite icin limit degerler: (-4.61175, 5.00625)
H20_Skoru icin limit degerler: (3.1499999999999986, 8.350000000000001)
O2_Skoru icin limit degerler: (-17.16, 47.120000000000005)
Azot_Skoru icin limit degerler: (2.05, 4.05)
H20_Oranı icin limit degerler: (-3.8500000000000005, 10.55)
O2_Oranı icin limit degerler: (-1.95, 4.05)
Azot_Oranı icin limit degerler: (-31.0625, 71.2775)
Kaynaklar icin limit degerler: (5.200000000000001, 9.999999999999998)
Isı_Oranı icin limit degerler: (-27.035714285714285, 57.82142857142857)


In [6]:
##IQR Degerlerini alt ve ust limit olarak donduren fonksiyon
def CalculateIQR(myList):
    q1_index = len(myList)/4
    #print(q1_index)
    if str(q1_index)[-2:] != ".0" :
        q1 = (myList[int(q1_index-0.5)] + myList[int(q1_index+0.5)])/2 
        q2 = (myList[int(3*q1_index-0.5)] + myList[int(3*q1_index+0.5)])/2 
    else:
        q1 = myList[int(q1_index)]
        q2 = myList[3*int(q1_index)] 


    IQR  = 1.5*(q2-q1)
    lower_limit =  q1 - IQR
    upper_limit =  q2 + IQR
    return (lower_limit , upper_limit)

In [7]:
#i sayisi sinirlara uyuyor mu kontrol ediyor
def isOK(i,bot,top):
        if (i < bot) or (i > top):
            return False
        else: 
            return True


In [9]:
# Polarite degeri icin ortalama:
availableSum = 0
avalibleCount = 0
unavailableSum = 0
unavailableCount = 0
for i in range(2,156):
    if (ws.cell(i,3).value != None) and (ws.cell(i,12).value == "yaşanabilir"):
        #print(ws.cell(i,3).value)
        availableSum = availableSum + ws.cell(i,3).value
        avalibleCount += 1
    elif (ws.cell(i,3).value != None) and (ws.cell(i,12).value == "yaşanamaz"):
        unavailableSum = unavailableSum + ws.cell(i,3).value
        unavailableCount += 1
print(f"Yasanabilir ortamlar icin ort polarite degeri: {availableSum/avalibleCount}")
print(f"Yasanamaz ortamlar icin ort polarite degeri:  {unavailableSum/unavailableCount}")
ortPolarite = (availableSum/avalibleCount) + (unavailableSum/unavailableCount)
print(f"ort polarite = {(availableSum+unavailableSum)/(avalibleCount+unavailableCount)}")

Yasanabilir ortamlar icin ort polarite degeri: -0.25542592592592606
Yasanamaz ortamlar icin ort polarite degeri:  0.14750104166666686
ort polarite = 0.0024473333333334085


In [10]:
## Uygun polarite degerlerinin yerlerine atanması
avaOrt = availableSum/avalibleCount
unavaOrt = unavailableSum/unavailableCount
for i in range(2,156):
    if (ws.cell(i,3).value == None):
        if ws.cell(i,12).value == "yaşanabilir":
            ws.cell(i,3).value = avaOrt
        else:
            ws.cell(i,3).value = unavaOrt
            
wb.save("Proje1_VeriKümesi.xlsx")

In [11]:
#h2o skoru için uygun ortalama veriler

h2availableSum = 0
h2avalibleCount = 0
h2unavailableSum = 0
h2unavailableCount = 0
lowerH2, upperH2 = CalculateIQR(all[3])
for i in range(2,156):
    if (ws.cell(i,4).value != None) and (ws.cell(i,12).value == "yaşanabilir"):
        if isOK(ws.cell(i,4).value , lowerH2, upperH2) == True:
            h2availableSum = h2availableSum + ws.cell(i,4).value
            h2avalibleCount += 1
        else:
            pass
    elif (ws.cell(i,4).value != None) and (ws.cell(i,12).value == "yaşanamaz"):
        if isOK(ws.cell(i,4).value , lowerH2, upperH2) == True:
            h2unavailableSum = h2unavailableSum + ws.cell(i,4).value
            h2unavailableCount += 1
        else:
            pass
print(f"Yasanabilir ortamlar icin ort h2oskor degeri: {h2availableSum/h2avalibleCount}")
print(f"Yasanamaz ortamlar icin ort h2oskor degeri:  {h2unavailableSum/h2unavailableCount}")
print(f"ort polarite = {(h2availableSum+h2unavailableSum)/(h2avalibleCount+h2unavailableCount)}")

Yasanabilir ortamlar icin ort h2oskor degeri: 5.951063829787231
Yasanamaz ortamlar icin ort h2oskor degeri:  5.7891304347826065
ort polarite = 5.843884892086329


In [12]:
##h2o skor icin eksik verilerin tamamlanması
h2avaOrt = h2availableSum/h2avalibleCount
h2unavaOrt = h2unavailableSum/h2unavailableCount
for i in range(2,156):
    if (ws.cell(i,4).value == None):
        if ws.cell(i,12).value == "yaşanabilir":
            ws.cell(i,4).value = avaOrt
        else:
            ws.cell(i,4).value = unavaOrt
            
wb.save("Proje1_VeriKümesi.xlsx")


In [13]:
#azot skoru ortalamaları
az_availableSum = 0
az_avalibleCount = 0
az_unavailableSum = 0
az_unavailableCount = 0
loweraz_, upperaz_ = CalculateIQR(all[5])
for i in range(2,156):
    if (ws.cell(i,6).value != None) and (ws.cell(i,12).value == "yaşanabilir"):
        if isOK(ws.cell(i,6).value , loweraz_, upperaz_) == True:
            az_availableSum = az_availableSum + ws.cell(i,6).value
            az_avalibleCount += 1
        else:
            pass
    elif (ws.cell(i,6).value != None) and (ws.cell(i,12).value == "yaşanamaz"):
        if isOK(ws.cell(i,6).value , loweraz_, upperaz_) == True:
            az_unavailableSum = az_unavailableSum + ws.cell(i,6).value
            az_unavailableCount += 1
        else:
            pass
print(f"Yasanabilir ortamlar icin ort azotskor degeri: {az_availableSum/az_avalibleCount}")
print(f"Yasanamaz ortamlar icin ort azotskor degeri:  {az_unavailableSum/az_unavailableCount}")
print(f"ort azotskor = {(az_availableSum+az_unavailableSum)/(az_avalibleCount+az_unavailableCount)}")

Yasanabilir ortamlar icin ort azotskor degeri: 2.806666666666667
Yasanamaz ortamlar icin ort azotskor degeri:  3.1533333333333333
ort azotskor = 3.037777777777778


In [14]:
avaOrt = az_availableSum/az_avalibleCount
unavaOrt = az_unavailableSum/az_unavailableCount
for i in range(2,156):
    if (ws.cell(i,6).value == None):
        if ws.cell(i,12).value == "yaşanabilir":
            ws.cell(i,6).value = avaOrt
        else:
            ws.cell(i,6).value = unavaOrt
            
wb.save("Proje1_VeriKümesi.xlsx")

In [15]:
h2oran_availableSum = 0
h2oran_avalibleCount = 0
h2oran_unavailableSum = 0
h2oran_unavailableCount = 0
lowerh2oran_, upperh2oran_ = CalculateIQR(all[6])
for i in range(2,156):
    if (ws.cell(i,7).value != None) and (ws.cell(i,12).value == "yaşanabilir"):
        if isOK(ws.cell(i,7).value , lowerh2oran_, upperh2oran_) == True:
            h2oran_availableSum = h2oran_availableSum + ws.cell(i,7).value
            h2oran_avalibleCount += 1
        else:
            pass
    elif (ws.cell(i,7).value != None) and (ws.cell(i,12).value == "yaşanamaz"):
        if isOK(ws.cell(i,7).value , lowerh2oran_, upperh2oran_) == True:
            h2oran_unavailableSum = h2oran_unavailableSum + ws.cell(i,7).value
            h2oran_unavailableCount += 1
        else:
            pass
print(f"Yasanabilir ortamlar icin ort H2O oran degeri: {h2oran_availableSum/h2oran_avalibleCount}")
print(f"Yasanamaz ortamlar icin ort H2O oran degeri:  {h2oran_unavailableSum/h2oran_unavailableCount}")
print(f"ort azotskor = {(h2oran_availableSum+h2oran_unavailableSum)/(h2oran_avalibleCount+h2oran_unavailableCount)}")

Yasanabilir ortamlar icin ort H2O oran degeri: 4.3533333333333335
Yasanamaz ortamlar icin ort H2O oran degeri:  3.554747474747475
ort azotskor = 3.804305555555556


In [16]:
avaOrt = h2oran_availableSum/h2oran_avalibleCount
unavaOrt = h2oran_unavailableSum/h2oran_unavailableCount
for i in range(2,156):
    if (ws.cell(i,7).value == None):
        if ws.cell(i,12).value == "yaşanabilir":
            ws.cell(i,7).value = avaOrt
        else:
            ws.cell(i,7).value = unavaOrt
            
wb.save("Proje1_VeriKümesi.xlsx")

In [17]:
o2oran_availableSum = 0
o2oran_avalibleCount = 0
o2oran_unavailableSum = 0
o2oran_unavailableCount = 0
lowero2oran_, uppero2oran_ = CalculateIQR(all[7])
for i in range(2,156):
    if (ws.cell(i,8).value != None) and (ws.cell(i,12).value == "yaşanabilir"):
        if isOK(ws.cell(i,8).value , lowero2oran_, uppero2oran_) == True:
            o2oran_availableSum = o2oran_availableSum + ws.cell(i,8).value
            o2oran_avalibleCount += 1
        else:
            pass
    elif (ws.cell(i,8).value != None) and (ws.cell(i,12).value == "yaşanamaz"):
        if isOK(ws.cell(i,8).value , lowero2oran_, uppero2oran_) == True:
            o2oran_unavailableSum = o2oran_unavailableSum + ws.cell(i,8).value
            o2oran_unavailableCount += 1
        else:
            pass
print(f"Yasanabilir ortamlar icin ort O2 oran degeri: {o2oran_availableSum/o2oran_avalibleCount}")
print(f"Yasanamaz ortamlar icin ort O2 oran degeri:  {o2oran_unavailableSum/o2oran_unavailableCount}")
print(f"ort o2 oran = {(o2oran_availableSum+o2oran_unavailableSum)/(o2oran_avalibleCount+o2oran_unavailableCount)}")

Yasanabilir ortamlar icin ort O2 oran degeri: 1.324
Yasanamaz ortamlar icin ort O2 oran degeri:  1.1372164948453607
ort o2 oran = 1.2007482993197278


In [18]:
avaOrt = o2oran_availableSum/o2oran_avalibleCount
unavaOrt = o2oran_unavailableSum/o2oran_unavailableCount
for i in range(2,156):
    if (ws.cell(i,8).value == None):
        if ws.cell(i,12).value == "yaşanabilir":
            ws.cell(i,8).value = avaOrt
        else:
            ws.cell(i,8).value = unavaOrt
            
wb.save("Proje1_VeriKümesi.xlsx")

In [19]:
kaynak_availableSum = 0
kaynak_avalibleCount = 0
kaynak_unavailableSum = 0
kaynak_unavailableCount = 0
lowerkaynak_, upperkaynak_ = CalculateIQR(all[9])
for i in range(2,156):
    if (ws.cell(i,10).value != None) and (ws.cell(i,12).value == "yaşanabilir"):
        if isOK(ws.cell(i,10).value , lowerkaynak_, upperkaynak_) == True:
            kaynak_availableSum = kaynak_availableSum + ws.cell(i,10).value
            kaynak_avalibleCount += 1
        else:
            pass
    elif (ws.cell(i,10).value != None) and (ws.cell(i,12).value == "yaşanamaz"):
        if isOK(ws.cell(i,10).value , lowerkaynak_, upperkaynak_) == True:
            kaynak_unavailableSum = kaynak_unavailableSum + ws.cell(i,10).value
            kaynak_unavailableCount += 1
        else:
            pass
print(f"Yasanabilir ortamlar icin ort kaynak degeri: {kaynak_availableSum/kaynak_avalibleCount}")
print(f"Yasanamaz ortamlar icin ort kaynak degeri:  {kaynak_unavailableSum/kaynak_unavailableCount}")
print(f"ort kaynak = {(kaynak_availableSum+kaynak_unavailableSum)/(kaynak_avalibleCount+kaynak_unavailableCount)}")

Yasanabilir ortamlar icin ort kaynak degeri: 7.487179487179487
Yasanamaz ortamlar icin ort kaynak degeri:  7.81395348837209
ort kaynak = 7.711999999999998


In [20]:
avaOrt = kaynak_availableSum/kaynak_avalibleCount
unavaOrt = kaynak_unavailableSum/kaynak_unavailableCount
for i in range(2,156):
    if (ws.cell(i,10).value == None):
        if ws.cell(i,12).value == "yaşanabilir":
            ws.cell(i,10).value = avaOrt
        else:
            ws.cell(i,10).value = unavaOrt
            
wb.save("Proje1_VeriKümesi.xlsx")

In [21]:
isi_availableSum = 0
isi_avalibleCount = 0
isi_unavailableSum = 0
isi_unavailableCount = 0
lowerisi_, upperisi_ = CalculateIQR(all[10])
for i in range(2,156):
    if (ws.cell(i,11).value != None) and (ws.cell(i,12).value == "yaşanabilir"):
        if isOK(ws.cell(i,11).value , lowerisi_, upperisi_) == True:
            isi_availableSum = isi_availableSum + ws.cell(i,11).value
            isi_avalibleCount += 1
        else:
            pass
    elif (ws.cell(i,11).value != None) and (ws.cell(i,12).value == "yaşanamaz"):
        if isOK(ws.cell(i,11).value , lowerisi_, upperisi_) == True:
            isi_unavailableSum = isi_unavailableSum + ws.cell(i,11).value
            isi_unavailableCount += 1
        else:
            pass
print(f"Yasanabilir ortamlar icin ort isi degeri: {isi_availableSum/isi_avalibleCount}")
print(f"Yasanamaz ortamlar icin ort isi degeri:  {isi_unavailableSum/isi_unavailableCount}")
print(f"ort isi = {(isi_availableSum+isi_unavailableSum)/(isi_avalibleCount+isi_unavailableCount)}")

Yasanabilir ortamlar icin ort isi degeri: 6.11212927947357
Yasanamaz ortamlar icin ort isi degeri:  17.985930948925276
ort isi = 13.780626190994461


In [22]:
avaOrt = isi_availableSum/isi_avalibleCount
unavaOrt = isi_unavailableSum/isi_unavailableCount
for i in range(2,156):
    if (ws.cell(i,11).value == None):
        if ws.cell(i,12).value == "yaşanabilir":
            ws.cell(i,11).value = avaOrt
        else:
            ws.cell(i,11).value = unavaOrt
            
wb.save("Proje1_VeriKümesi.xlsx")